In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import json
import numpy as np
import matplotlib.pyplot as plt 
import rasterio
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
import os
# os.environ['CUDA_LAUNCH_BLOCKING']='1'
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Final Approach: Mask2Former

In [ ]:
pip install -U accelerate

In [3]:
import torch
# import torch.multiprocessing as mp
# mp.set_start_method('spawn', force=True)

In [ ]:
pip install -U pytorch_lightning

In [4]:
import torch
from transformers import Mask2FormerImageProcessor#, Mask2FormerConfig
type_dict = {0:'blood_vessel', 1:'glomerulus', 2:'unsure'}

# load Mask2Former fine-tuned on COCO instance segmentation

# model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-tiny-cityscapes-instance", 
#                                                             id2label=type_dict,
#                                                             ignore_mismatched_sizes=True)


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
torch.cuda.empty_cache() 
import gc
gc.collect()

In [ ]:
# processor = Mask2FormerImageProcessor(reduce_labels=True,
#                                                       ignore_index=255,
#                                                       do_resize=False, 
#                                                       do_rescale=False, 
#                                                       do_normalize=False)
# configuration = Mask2FormerConfig(id2label=type_dict)
# model = Mask2FormerForUniversalSegmentation(configuration)

In [ ]:
# train = pd.read_csv("/kaggle/input/hubmap-train-test/train_set.csv")
# val = pd.read_csv("/kaggle/input/hubmap-train-test/val_set.csv")
# train_gr = train.groupby('0').agg(lambda x: list(x))
# val_gr = val.groupby('0').agg(lambda x: list(x))
train_gr = pd.read_csv("/kaggle/input/hubmap-train-test/train_grouped.csv")
val_gr = pd.read_csv("/kaggle/input/hubmap-train-test/val_grouped.csv")


In [ ]:
train_gr

In [ ]:
"""
Code to visualize images
"""
# import ast
# item = train_gr.iloc[0] 
# image = rasterio.open(item.path).read()
# type_ = ast.literal_eval(item['structure_types'])
# coords = ast.literal_eval(item['coordinates'])
# image_ = torch.from_numpy(image).float() / 255.0  # Normalize image to [0, 1]
# image_shape = image.shape[1:]
# mask = np.zeros(image_shape, dtype=np.uint8)
# pts_array = []
# obj_cnt = 1
# type_dict = {'blood_vessel':1, 'glomerulus':2, 'unsure':3}
# inst2cls ={0:0}
# print(type(coords))
# for typ, coord in zip(type_, coords):
#     pts = np.array(coord).reshape((-1, 1, 2)).astype(np.int32)
# #     print(pts)
#     cv2.fillPoly(mask, [pts], obj_cnt)
#     inst2cls[obj_cnt] = type_dict[typ]
#     obj_cnt+=1
# #     break
# image = image.transpose(1,2,0)
# #     print()
# #     continue
# arr = [0,0,0]
# # Overlay the mask on the image
# overlay = np.copy(image)
# print(arr)
# overlay[mask > 0] =  (1,0,0)# Set the mask pixels to red (adjust color as needed)

# # Plot the image with the overlay
# fig, axes = plt.subplots(1, 3, figsize=(10, 5))
# axes[0].imshow(image)
# axes[0].set_title('Image')
# axes[1].imshow(mask)
# axes[1].set_title('Image with Mask Overlay')
# axes[2].imshow(overlay)
# axes[2].set_title('Image with Mask Overlay')
# plt.show()

In [ ]:
# segmentation_map = np.where(mask == 0, 255, mask - 1)

In [ ]:
# inputs = processor([image], [mask], instance_id_to_semantic_id=inst2cls, return_tensors="pt")
# inputs = {k: v.squeeze() if isinstance(v, torch.Tensor) else v[0] for k,v in inputs.items()}


In [ ]:
"""
Pytorch Datasets

"""
import ast
class CustomDataset_instances:
    """
    Instance-wise Segmentation
    """
    def __init__(self, df, transform):
        self.df = df
        self.processor = Mask2FormerImageProcessor.from_pretrained("facebook/mask2former-swin-tiny-cityscapes-instance",
                                                      reduce_labels=True,
                                                      ignore_index=255,
                                                      do_resize=False, 
                                                      do_rescale=False, 
                                                      do_normalize=False)
        self.transform = transform
    def __getitem__(self, index):
        # Load image
        item = self.df.iloc[index]
        image = rasterio.open(item.path).read()
        type_ = ast.literal_eval(item['structure_types'])
        coords = ast.literal_eval(item['coordinates'])
#         image = image.astype(float) / 255.0  # Normalize image to [0, 1]
        image_shape = image.shape[1:]
        mask = np.zeros(image_shape, dtype=np.uint8)
        pts_array = []
        obj_cnt = 1
        type_dict = {'blood_vessel':0, 'glomerulus':1, 'unsure':2}
        inst2cls = {0:0}
        for typ, coord in zip(type_, coords):
            if typ == 'unsure':
                continue
            else:
                pts = np.array(coord).reshape((-1, 1, 2)).astype(np.int32)
                cv2.fillPoly(mask, [pts], obj_cnt)
                inst2cls[obj_cnt] = type_dict[typ]+1
                obj_cnt+=1
        if self.transform is not None:
            transformed = self.transform(image=image.transpose(1,2,0), mask=mask)
            image, mask = transformed['image'], transformed['mask']
            image = image.transpose(2,0,1)
        if len(inst2cls.keys())==1:
            # Some image does not have annotation (all ignored)
            inputs = self.processor([image], return_tensors="pt")
            inputs = {k:v.squeeze() for k,v in inputs.items()}
            inputs["class_labels"] = torch.tensor([3])
            inputs["mask_labels"] = torch.zeros((0, inputs["pixel_values"].shape[-2], inputs["pixel_values"].shape[-1]))
            return inputs
        inputs = self.processor([image], [mask], instance_id_to_semantic_id=inst2cls, return_tensors="pt", reduce_labels=True)
        inputs = {k: v.squeeze() if isinstance(v, torch.Tensor) else v[0] for k,v in inputs.items()}
#         print(inputs['class_labels'])
        return inputs

    def __len__(self):
        return len(self.df)


In [ ]:
import albumentations as A

ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
ADE_STD = np.array([58.395, 57.120, 57.375]) / 255

# note that you can include more fancy data augmentation methods here
train_transform = A.Compose([
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),
])

In [ ]:
def collate_fn(batch):
    pixel_values = torch.stack([example["pixel_values"] for example in batch])
    pixel_mask = torch.stack([example["pixel_mask"] for example in batch])
    class_labels = [example["class_labels"] for example in batch]
    mask_labels = [example["mask_labels"] for example in batch]
    return {"pixel_values": pixel_values, "pixel_mask": pixel_mask, "class_labels": class_labels, "mask_labels": mask_labels}


In [ ]:
# from transformers import Trainer, TrainingArguments
# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=3,              # total number of training epochs
#     per_device_train_batch_size=16,  # batch size per device during training
#     per_device_eval_batch_size=64,   # batch size for evaluation
#     warmup_steps=100,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,
# )

In [ ]:
train_dataset = CustomDataset_instances(train_gr, train_transform)
val_dataset = CustomDataset_instances(val_gr, train_transform)

### Check the dataloaders once

In [ ]:
samp_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)


In [ ]:
item = next(iter(samp_dataloader))

In [ ]:
item

In [ ]:
im = item['pixel_mask'][1].numpy()
plt.imshow(im)

In [ ]:
im = item['mask_labels'][1].numpy()
plt.imshow(im[3])

In [ ]:
item['class_labels']

In [ ]:
im[0]

In [ ]:
item['pixel_mask'].shape

In [ ]:
item['mask_labels'][0].shape

In [ ]:
model.train()
output = model(pixel_values = item['pixel_values'],
                mask_labels = item["mask_labels"],
                class_labels = item["class_labels"],
     )

In [ ]:
output.loss

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)


In [ ]:
len(train_dataloader)

In [ ]:
len(val_dataloader)

In [ ]:
torch.backends.cudnn.enabled = False


### Training pytorch_lightning

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from pytorch_lightning.callbacks import LearningRateMonitor
lr_monitor = LearningRateMonitor(logging_interval='step')

class Mask2Former(pl.LightningModule):
    def __init__(self):
        super().__init__()
        from transformers import Mask2FormerForUniversalSegmentation
        self.type_dict = {0:'blood_vessel', 1:'glomerulus', 2:'unsure'}
        self.model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-tiny-cityscapes-instance", 
                                                            id2label=self.type_dict,
                                                            ignore_mismatched_sizes=True)

        self.learning_rate = 5e-5

    def forward(self, pixel_values, mask_labels, class_labels):
        return self.model(pixel_values, mask_labels, class_labels)

    def training_step(self, batch, batch_idx):
        outputs = self(
            pixel_values=batch["pixel_values"],
            mask_labels=[labels for labels in batch["mask_labels"]],
            class_labels=[labels for labels in batch["class_labels"]],
        )
        loss = outputs.loss
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(
            pixel_values=batch["pixel_values"],
            mask_labels=[labels for labels in batch["mask_labels"]],
            class_labels=[labels for labels in batch["class_labels"]],
        )
        val_loss = outputs.loss
        self.log('val_loss', val_loss, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=0.2)

        lr_scheduler_cont = {
            'scheduler':  torch.optim.lr_scheduler.ReduceLROnPlateau(
                                                                        optimizer,
                                                                        mode='min',
                                                                        factor=0.8,
                                                                        patience=50,
                                                                        threshold=1e-2,
                                                                        min_lr=1e-8,
                                                                        verbose=True
                                                                    ),
            'name': 'learning_rate',
            'monitor' : 'train_loss',
            "interval" : "step" 
        }
        return optimizer


In [ ]:
model_m2f = Mask2Former()
wandb_logger = WandbLogger()
checkpoint_callback = ModelCheckpoint(dirpath="/kaggle/working/ckpt/", save_top_k=3, monitor="val_loss")
trainer = pl.Trainer(max_epochs=50, devices='auto',accelerator="gpu",precision=16, logger=wandb_logger, callbacks=[checkpoint_callback])  # Set the appropriate number of GPUs
trainer.fit(model_m2f, train_dataloader, val_dataloader)


In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from pytorch_lightning.callbacks import LearningRateMonitor

In [ ]:
class Mask2Former_pred(pl.LightningModule):
    def __init__(self):
        super().__init__()
        from transformers import Mask2FormerForUniversalSegmentation
        self.type_dict = {0:'blood_vessel', 1:'glomerulus', 2:'unsure'}
        self.model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-tiny-cityscapes-instance", 
                                                            id2label=self.type_dict,
                                                            ignore_mismatched_sizes=True)

        self.learning_rate = 5e-5

    def forward(self, pixel_values, mask_labels, class_labels):
        return self.model(pixel_values, mask_labels, class_labels)

    def training_step(self, batch, batch_idx):
        outputs = self(
            pixel_values=batch["pixel_values"],
            mask_labels=[labels for labels in batch["mask_labels"]],
            class_labels=[labels for labels in batch["class_labels"]],
        )
        loss = outputs.loss
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(
            pixel_values=batch["pixel_values"],
            mask_labels=[labels for labels in batch["mask_labels"]],
            class_labels=[labels for labels in batch["class_labels"]],
        )
        val_loss = outputs.loss
        self.log('val_loss', val_loss, on_step=False, on_epoch=True)
        
    def predict_step(self, batch, batch_idx):
        print(batch)
        outputs = self.model(
            pixel_values=batch["pixel_values"],
        )
        return outputs

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=0.2)

        lr_scheduler_cont = {
            'scheduler':  torch.optim.lr_scheduler.ReduceLROnPlateau(
                                                                        optimizer,
                                                                        mode='min',
                                                                        factor=0.8,
                                                                        patience=50,
                                                                        threshold=1e-2,
                                                                        min_lr=1e-8,
                                                                        verbose=True
                                                                    ),
            'name': 'learning_rate',
            'monitor' : 'train_loss',
            "interval" : "step" 
        }
        return optimizer



  

In [ ]:
type_dict = {0:'blood_vessel', 1:'glomerulus', 2:'unsure'}
from transformers import Mask2FormerForUniversalSegmentation
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-tiny-cityscapes-instance", 
                                                            id2label=type_dict,
                                                            ignore_mismatched_sizes=True)

In [ ]:
import ast
class CustomDataset_pred:
    def __init__(self, df, transform):
        self.df = df
        self.processor = Mask2FormerImageProcessor.from_pretrained("facebook/mask2former-swin-tiny-cityscapes-instance",
                                                      reduce_labels=True,
                                                      ignore_index=255,
                                                      do_resize=False, 
                                                      do_rescale=False, 
                                                      do_normalize=False)
        self.transform = transform
    def __getitem__(self, index):
        # Load image
        item = self.df[index]
        image = rasterio.open(item).read()
        
        if self.transform is not None:
            transformed = self.transform(image=image.transpose(1,2,0))
            image= transformed['image']
            image = image.transpose(2,0,1)
            
        inputs = self.processor([image], return_tensors="pt",reduce_labels=True)
        inputs = {k: v.squeeze() if isinstance(v, torch.Tensor) else v[0] for k,v in inputs.items()}
        return inputs

    def __len__(self):
        return len(self.df)


In [ ]:
pred_dataset = CustomDataset_pred(['/kaggle/input/hubmap-hacking-the-human-vasculature/test/72e40acccadf.tif'], train_transform)
samp_dataloader = torch.utils.data.DataLoader(pred_dataset, batch_size=1, shuffle=True)


In [ ]:
chk = torch.load("/kaggle/working/ckpt/epoch=14-step=2460.ckpt", map_location=torch.device('cpu'))

In [ ]:
model = model.load_state_dict(chk["state_dict"])

In [ ]:
model_pred = Mask2Former_pred.load_from_checkpoint("/kaggle/working/ckpt/epoch=14-step=2460.ckpt")

In [ ]:
inputs

In [ ]:
trainer_pred = pl.Trainer(devices='auto',accelerator="cpu",precision=16)  # Set the appropriate number of GPUs

res = trainer_pred.predict(model_pred, samp_dataloader)

In [ ]:
res

In [ ]:
pred_instance_map = processor.post_process_instance_segmentation(
    res[0], target_sizes=[(512,512)], return_binary_maps=True

)[0]

In [ ]:
plt.imshow(pred_instance_map[0]['segmentation'][0])

In [ ]:
np.unique(pred_instance_map['segmentation'])

In [ ]:
img= cv2.cvtColor(cv2.imread('/kaggle/input/hubmap-hacking-the-human-vasculature/test/72e40acccadf.tif'), cv2.COLOR_BGR2RGB) #return np array of pixels 
plt.figure()
plt.imshow(img)
plt.title("Image")
plt.show()

# predicted_semantic_map = processor.post_process_instance_segmentation(outputs, target_sizes=[(512,512)])[0]
# color_segmentation_map = pred_instance_map['segmentation']
overlay = np.zeros([512,512])
for d in pred_instance_map['segments_info']:
    print("hi")
    if d['score']>0.7:
        plt.imshow(pred_instance_map['segmentation'][d['id']])
        
        plt.show()
        overlay+=pred_instance_map['segmentation'][d['id']].numpy()
#         color_segmentation_map[color_segmentation_map==d['id']] = 50+50*d['label_id']
#         overlay[color_segmentation_map==d['id']] = 50+50*d['label_id']
#         plt.imshow(overlay)
        
#     else:
#         color_segmentation_map[color_segmentation_map==d['id']] = 0
#     plt.imshow(color_segmentation_map, alpha=0.6)
# #     color_segmentation_map = color_segmentation_map.abs()
# #     print(color_segmentation_map)
plt.imshow(img.reshape(512,512,3))
plt.imshow(overlay, alpha=0.6)
# plt.title("Predictions")
# plt.show()

In [ ]:
overlay.shape, pred_instance_map['segmentation'][d['id']].shape

In [ ]:
with torch.no_grad():
    outputs = model_m2f.model(pixel_values = torch.from_numpy(np.array(inputs['pixel_values'])))

In [ ]:
class_queries_logits = outputs.class_queries_logits
masks_queries_logits = outputs.masks_queries_logits

# Perform post-processing to get instance segmentation map
pred_instance_map = processor.post_process_semantic_segmentation(
    outputs, target_sizes=[(512,512)]

)[0]

In [ ]:
pred_instance_map

In [ ]:
plt.imshow(image.transpose(1,2,0))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(pred_instance_map)

### Manual Training loop

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

running_loss = 0.0
num_samples = 0
num_epochs = 50
for epoch in range(num_epochs):
    print("Epoch:", epoch)
    running_loss = 0.0
    pbar = tqdm(train_dataloader)
    model.train()
    for idx, batch in enumerate(pbar):
        # Reset the parameter gradients
        optimizer.zero_grad()
#         print(batch["class_labels"])
        # Forward pass
        outputs = model(
                pixel_values=batch["pixel_values"].cuda(),
#                 pixel_mask = batch["pixel_mask"].cuda(),
                mask_labels=[labels.cuda() for labels in batch["mask_labels"]],
                class_labels=[labels.cuda() for labels in batch["class_labels"]],
        )

        # Backward propagation
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        pbar.set_postfix({'loss':(running_loss/(idx+1))})
        
    # Print training loss for the epoch
    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}")

    # Evaluation on the validation set
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_dataloader):
#             images = batch['image']
            outputs = model(
                    pixel_values=batch["pixel_values"].cuda(),
#                     pixel_mask = batch["pixel_mask"].cuda(),
                    mask_labels=[labels.cuda() for labels in batch["mask_labels"]],
                    class_labels=[labels.cuda() for labels in batch["class_labels"]],
            )

            # Calculate the loss
            loss = outputs.loss
            val_loss += loss.item()
            
    # Print validation loss for the epoch
    val_loss /= len(val_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}")

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
model
optimizer = torch.optim.Adam(model.parameters(), lr=0.0025)

running_loss = 0.0
num_samples = 0
num_epochs = 50
for epoch in range(num_epochs):
    print("Epoch:", epoch)
    running_loss = 0.0
    pbar = tqdm(train_dataloader)
    model.train()
    for idx, batch in enumerate(pbar):
        # Reset the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(
                pixel_values=batch["pixel_values"],
#                 pixel_mask = batch["pixel_mask"].cuda(),
                mask_labels=[labels for labels in batch["mask_labels"]],
                class_labels=[labels for labels in batch["class_labels"]],
        )

        # Backward propagation
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        pbar.set_postfix({'loss':(running_loss/(idx+1))})
        
    # Print training loss for the epoch
    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}")

    # Evaluation on the validation set
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_dataloader):
#             images = batch['image']
            outputs = model(
                    pixel_values=batch["pixel_values"],
#                     pixel_mask = batch["pixel_mask"].cuda(),
                    mask_labels=[labels for labels in batch["mask_labels"]],
                    class_labels=[labels for labels in batch["class_labels"]],
            )

            # Calculate the loss
            loss = outputs.loss
            val_loss += loss.item()
            
    # Print validation loss for the epoch
    val_loss /= len(val_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}")

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k,v.shape)
    else:
        print(k,len(v))

In [ ]:
print([label.shape for label in batch["class_labels"]])


In [ ]:
print([label.shape for label in batch["mask_labels"]])


In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
model(
        pixel_values=batch["pixel_values"].cuda(),
        pixel_mask = batch["pixel_mask"].cuda(),
        mask_labels=[labels.cuda() for labels in batch["mask_labels"]],
        class_labels=[labels.cuda() for labels in batch["class_labels"]],
)

In [ ]:
# trainer.train()


## Previous failed approaches

In [ ]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install 'git+https://github.com/facebookresearch/detectron2.git'


In [ ]:
# clone and install Mask2Former
!git clone https://github.com/facebookresearch/Mask2Former.git
%cd Mask2Former
!pip install -U opencv-python
!pip install git+https://github.com/cocodataset/panopticapi.git
!pip install -r requirements.txt
%cd mask2former/modeling/pixel_decoder/ops
!python setup.py build install
%cd ../../../../

In [ ]:
annotation = mmengine.load('./ballondatasets/balloon/train/via_region_data.json')


In [ ]:
!pip install openmim
!mim install git+https://github.com/open-mmlab/mmengine.git
!mim install git+https://github.com/open-mmlab/mmcv.git
!mim install git+https://github.com/open-mmlab/mmdetection.git
!pip install -U numpy


In [ ]:
!pip install -qqq /kaggle/input/mmdetectron-31-wheel/*.whl
!pip install -U numpy
# !pip install git+https://github.com/open-mmlab/mmengine.git
# !pip install git+https://github.com/open-mmlab/mmcv.git
# !pip install git+https://github.com/open-mmlab/mmdetection.git


In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
# Check Pytorch installation
import torch, torchvision
print("torch version:",torch.__version__, "cuda:",torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print("mmdetection:",mmdet.__version__)

# Check mmcv installation
import mmcv
print("mmcv:",mmcv.__version__)

# Check mmengine installation
import mmengine
print("mmengine:",mmengine.__version__)

In [ ]:
pip install -q git+https://github.com/qubvel/segmentation_models.pytorch


In [ ]:
pip install -q pycocotools

In [ ]:
%cd ../
!zip -r all_req.zip wheels


In [ ]:
import segmentation_models_pytorch as smp


In [ ]:

# Load the image data
# image =  rasterio.open("/kaggle/input/hubmap-hacking-the-human-vasculature/train/0006ff2aa7cd.tif").read().transpose(1,2,0)
# # image = image.reshape((512, 512))?
# plt.imshow(image)

In [ ]:
def get_annotations():
    annotations = []

    with open("/kaggle/input/hubmap-hacking-the-human-vasculature/polygons.jsonl", "r") as file:
        for line in tqdm(file):
            annotation = json.loads(line)
            annotations.append(annotation)
    return annotations

def get_instance_tuples(annotations):
    image_tup = []

    for annotation in annotations:
        image_id = annotation["id"]
    #     image_ids.append(image_id)

        for structure_annotation in annotation["annotations"]:
            image_path = f"/kaggle/input/hubmap-hacking-the-human-vasculature/train/{image_id}.tif"
            structure_type = structure_annotation["type"]
    #         structure_types.append(structure_type)

            coordinates = structure_annotation["coordinates"]
    #         coordinates_list.append(coordinates)
            image_tup.append((image_path, structure_type, coordinates))
    return image_tup

def get_semantic_dicts(annotations):
    image_dict={}

    for annotation in annotations:
        image_id = annotation["id"]
        image_path = f"/kaggle/input/hubmap-hacking-the-human-vasculature/train/{image_id}.tif"
        image_dict[image_id] = {'path':image_path, 'structure_types' : [], 'coordinates' : []}
        for structure_annotation in annotation["annotations"]:
            structure_type = structure_annotation["type"]
            image_dict[image_id]['structure_types'].append(structure_type)
            coordinates = structure_annotation["coordinates"]
            image_dict[image_id]['coordinates'].append(coordinates)
    return image_dict

In [ ]:
annotations = get_annotations()
image_dict = get_semantic_dicts(annotations)
image_tup = get_instance_tuples(annotations)

In [ ]:
all_data = pd.DataFrame(image_dict).T

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(all_data,test_size=0.2 )

In [ ]:
train_df.to_csv("train_grouped.csv", index=False)
val_df.to_csv("val_grouped.csv", index=False)

In [ ]:
!mim download mmdet --config mask2former_swin-s-p4-w7-224_8xb2-lsj-50e_coco --dest ./checkpoints

In [ ]:
import mmengine

annotation = mmengine.load('/kaggle/input/hubmap-hacking-the-human-vasculature/polygons.jsonl')

In [ ]:
train = pd.read_csv("/kaggle/input/hubmap-train-test/train_set.csv")
val = pd.read_csv("/kaggle/input/hubmap-train-test/val_set.csv")
train_gr = train.groupby('0').agg(lambda x: list(x))
val_gr = val.groupby('0').agg(lambda x: list(x))

In [ ]:
train

In [ ]:
import ast
instance_seg = np.array(ast.literal_eval(train.iloc[0]['2']))[..., 1]
class_id_map = np.array(ast.literal_eval(train.iloc[0]['2']))[..., 0]
class_labels = np.unique(class_id_map)

In [ ]:
instance_seg

In [ ]:
class_id_map

In [ ]:
class_labels

In [ ]:
# train = train.to_records(index=False).tolist()
# val = val.to_records(index=False).tolist()

In [ ]:
train_gr = train.groupby('0').agg(lambda x: list(x))
val_gr = val.groupby('0').agg(lambda x: list(x))

In [ ]:
train_gr

In [ ]:
len(ast.literal_eval(train_gr.iloc[0]['2'][0]))

In [ ]:
import ast
np.array(ast.literal_eval(train_gr.iloc[0]['2'][0])[0]).transpose(1,0)
# del px
# del py

In [ ]:
import ast 
def get_coco(df, out_file):
    type_dict = {'blood_vessel':0, 'glomerulus':1, 'unsure':2}
    images = []
    annotations = []
    obj_count = 0
    idx = 0
    for i, row in df.iterrows():
#         print(row)
        file_path = i
        filename = file_path.split('/')[-1]
        images.append(dict(
                    id=idx,
                    file_name=filename,
                    height=512,
                    width=512))
        idx += 1
        bboxes = []
        labels = []
        masks = []
        for type_, coords in zip(row['1'], row['2']):
            if type_ == 'unsure':
                continue
            coords = ast.literal_eval(coords)
            for coord in coords:
                poly = [(x + 0.5, y + 0.5) for x, y in coord]
                poly = [p for x in poly for p in x]
                px, py = np.array(coord).transpose(1,0)
                x_min, y_min, x_max, y_max = (
                    min(px), min(py), max(px), max(py))


                data_anno = dict(
                    image_id=idx,
                    id=obj_count,
                    category_id=type_dict[type_],
                    bbox=[x_min, y_min, x_max - x_min, y_max - y_min],
                    area=(x_max - x_min) * (y_max - y_min),
                    segmentation=[poly],
                    iscrowd=0)
                annotations.append(data_anno)
                obj_count += 1

    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=[{'id':0, 'name': 'blood_vessel'}, {'id':1, 'name': 'glomerulus'}])
    mmengine.dump(coco_format_json, out_file)


In [ ]:
!rm -rf /kaggle/working/json_configs/

In [ ]:
get_coco(train_gr, '/kaggle/working/json_configs/train.json')
get_coco(val_gr, '/kaggle/working/json_configs/val.json')

In [ ]:
from mmengine import Config
cfg = Config.fromfile('/kaggle/working/checkpoints/mask2former_swin-s-p4-w7-224_8xb2-lsj-50e_coco.py')


In [ ]:
from pprint import pprint
pprint(cfg)

In [ ]:
import os
os.listdir("/kaggle/working/checkpoints/")

In [ ]:
# from mmengine.runner import set_random_seed

# Modify dataset classes and color
cfg.metainfo = {
    'classes': (0, 1),
    'palette': [
        (220, 20, 60),
        (20, 220, 120)
    ]
}

# Modify dataset type and path
cfg.data_root = '' #'/kaggle/input/hubmap-hacking-the-human-vasculature/train'

cfg.train_dataloader.dataset.ann_file = '/kaggle/working/json_configs/train.json'
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix.img = '/kaggle/input/hubmap-hacking-the-human-vasculature/train'
cfg.train_dataloader.dataset.metainfo = cfg.metainfo

cfg.val_dataloader.dataset.ann_file = '/kaggle/working/json_configs/val.json'
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix.img = '/kaggle/input/hubmap-hacking-the-human-vasculature/train'
cfg.val_dataloader.dataset.metainfo = cfg.metainfo

cfg.test_dataloader = cfg.val_dataloader

# Modify metric config
cfg.val_evaluator.ann_file = '/kaggle/working/json_configs/val.json'
cfg.test_evaluator = cfg.val_evaluator

# Modify num classes of the model in box head and mask head
# cfg.model.roi_head.bbox_head.num_classes = 2
# cfg.model.roi_head.mask_head.num_classes = 2
cfg.num_classes = 2
cfg.num_things_classes =2
# We can still the pre-trained Mask RCNN model to obtain a higher performance
cfg.load_from = '/kaggle/working/checkpoints/mask2former_swin-s-p4-w7-224_8xb2-lsj-50e_coco_20220504_001756-c9d0c4f2.pth'

# Set up working dir to save files and logs.
cfg.work_dir = '/kaggle/working/exps'


# We can set the evaluation interval to reduce the evaluation times
cfg.train_cfg.val_interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.default_hooks.checkpoint.interval = 3

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optim_wrapper.optimizer.lr = 1e-3
cfg.default_hooks.logger.interval = 10


# Set seed thus the results are more reproducible
# cfg.seed = 0
# set_random_seed(0, deterministic=False)

# We can also use tensorboard to log the training process
cfg.visualizer.vis_backends.append({"type":'TensorboardVisBackend'})

In [ ]:
pprint(cfg.train_dataloader)

In [ ]:
!pip install requests==2.28.2

In [ ]:
# from mmdet.datasets import build_dataset
# from mmdet.models import build_detector
from mmengine.runner import Runner

# build the runner from config
runner = Runner.from_cfg(cfg)

In [ ]:
runner.train()

# UNET

In [ ]:
from sklearn.model_selection import train_test_split
train_tup, test_tup = train_test_split(image_tup,test_size=0.2 )

In [ ]:
len(test_tup)

In [ ]:
# tile_meta = pd.read_csv("/kaggle/input/hubmap-hacking-the-human-vasculature/tile_meta.csv")
# train_images = os.listdir("/kaggle/input/hubmap-hacking-the-human-vasculature/train/")
# tile_meta_available = tile_meta[tile_meta['dataset'].isin([1,2])]
# train_images_filtered = []
# for img in train_images:
#     img_name = img.split("/")[-1].split(".")[0]
# #     print(img_name)
#     if img_name in tile_meta_available['id'].tolist():
#         train_images_filtered.append(img)

In [ ]:
# for k, v in image_dict.items():
#     image =  rasterio.open(v['path']).read()
#     image_shape = image.shape[1:]
#     mask = np.zeros(image_shape, dtype=np.uint8)
#     pts_array = []
#     for coord in v['coordinates']:
#         pts = np.array(coord).reshape((-1, 1, 2)).astype(np.int32)
#         pts_array.append(pts)
#     cv2.fillPoly(mask, pts_array, 255)
# #     mask = np.expand_dims(mask.astype(np.float32), axis=0)
#     print(mask.shape)
#     overlay = np.copy(image.transpose(1,2,0))
#     overlay[mask > 0] = [1,0,0]  # Set the mask pixels to red (adjust color as needed)

#     # Plot the image with the overlay
#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))
#     axes[0].imshow(image.transpose(1,2,0))
#     axes[0].set_title('Image')
#     axes[1].imshow(overlay)
#     axes[1].set_title('Image with Mask Overlay')
#     plt.show()
#     break
# #     np.expand_dims(mask.astype(np.float32), axis=0)

In [ ]:
# for annotation in annotations:
# #     print(annotation)
# #     break
#     id_ = annotation['id']
#     image =  rasterio.open(f"/kaggle/input/hubmap-hacking-the-human-vasculature/train/{id_}.tif").read().transpose(1,2,0)
#     for i in range(len(annotation["annotations"])):
#         structure_type = annotation["annotations"][i]["type"]
#         coordinates = annotation["annotations"][i]["coordinates"]
#         # Create a binary mask from the coordinates
#         mask = np.zeros_like(image[..., 0], dtype=np.uint8)
#         pts = np.array(coordinates).reshape((-1, 1, 2)).astype(np.int32)
#         cv2.fillPoly(mask, [pts], 255)

#         # Overlay the mask on the image
#         overlay = cv2.addWeighted(image, 0.5, cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB), 0.5, 0)

#         # Display the image with the overlay
#         plt.imshow(overlay)
#         plt.title(f"Structure Type: {structure_type}")
#         plt.show()

In [ ]:
preprocess_input = get_preprocessing_fn('timm-resnest101e', pretrained='imagenet')

In [ ]:
image =  rasterio.open("/kaggle/input/hubmap-hacking-the-human-vasculature/train/0006ff2aa7cd.tif").read()
preprocess_input(image.transpose(1,2,0))

In [ ]:
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import ast
import torch
from torch.utils.data import Dataset

class CustomDataset_instance(Dataset):
    def __init__(self, image_tup):
        self.image_tup = image_tup
        self.preprocess_input = get_preprocessing_fn('timm-resnest101e', pretrained='imagenet')

    def __getitem__(self, index):
        # Load image
        image_path, type_, coords, = self.image_tup[index]
        image = rasterio.open(image_path).read()#.transpose(1, 2, 0)
        image = self.preprocess_input(image.transpose(1,2,0)).transpose(2,0,1)
#         image = torch.from_numpy(image).float() / 255.0  # Normalize image to [0, 1]
        type_dict = {'blood_vessel':1, 'glomerulus':2, 'unsure':3}
        # Load corresponding annotation
#         annotation = self.annotations[index]
        mask = self._create_mask(ast.literal_eval(coords), image.shape[1:], type_dict[type_])

        return {'image': image, 'mask': mask}

    def __len__(self):
        return len(self.image_tup)

    def _create_mask(self, coordinates, image_shape, type_):
        mask = np.zeros(image_shape, dtype=np.uint8)
#         print(mask.shape)
        pts = np.array(coordinates).reshape((-1, 1, 2)).astype(np.int32)
        cv2.fillPoly(mask, [pts], type_)
#         print((mask > 0).astype(np.uint8).shape)
        return np.expand_dims(mask.astype(np.float32), axis=0)
        
class CustomDataset_semantic(Dataset):
    def __init__(self, image_dict):
        self.image_dict = image_dict
        self.keys = list(image_dict.keys())

    def __getitem__(self, index):
        # Load image
        key = self.keys[index]
        value = self.image_dict[key]
        
        image = rasterio.open(value['path']).read()#.transpose(1, 2, 0)
        image = torch.from_numpy(image).float() / 255.0  # Normalize image to [0, 1]
        image_shape = image.shape[1:]
        mask = np.zeros(image_shape, dtype=np.uint8)
        pts_array = []
        for coord in (value['coordinates']):
            pts = np.array(coord).reshape((-1, 1, 2)).astype(np.int32)
            pts_array.append(pts)
        cv2.fillPoly(mask, pts_array, 1)
        mask = np.expand_dims(mask.astype(np.float32), axis=0)
        return {'image': image, 'mask': mask}

    def __len__(self):
        return len(self.keys)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:
# combined_
train_dataset = CustomDataset_instance(train)
test_dataset = CustomDataset_instance(val)

# dataset_2 = CustomDataset_semantic(image_dict)
# dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [ ]:

dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
data = next(iter(dataloader))

In [ ]:
test_dataset

In [ ]:
pd.DataFrame(train_tup).to_csv("train_set.csv", index=False)
pd.DataFrame(test_tup).to_csv("val_set.csv", index=False)

In [ ]:
data = next(iter(dataloader))

In [ ]:
# data['mask']

In [ ]:
c=0
for data in dataloader:
    
    image = data['image'].squeeze().numpy().transpose(1,2,0)
    mask = data['mask'].squeeze().numpy()
#     print()
#     continue
    arr = [0,0,0]
    # Overlay the mask on the image
    overlay = np.copy(image)
    label = np.unique(mask)[np.nonzero(np.unique(mask))][0]
    arr[int(label-1)] = 1
    print(arr)
    overlay[mask > 0] =  arr # Set the mask pixels to red (adjust color as needed)

    # Plot the image with the overlay
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(image)
    axes[0].set_title('Image')
    axes[1].imshow(overlay)
    axes[1].set_title('Image with Mask Overlay')
    plt.show()

In [ ]:
# data['image'].shape

In [ ]:
# class UNet(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(UNet, self).__init__()

#         # Encoder
#         self.encoder = nn.Sequential(
#             nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(64, 64, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )

#         # Decoder
#         self.decoder = nn.Sequential(
#             nn.Conv2d(64, 64, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(64, out_channels, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
#         )

#     def forward(self, x):
#         # Encoder
#         x = self.encoder(x)

#         # Decoder
#         x = self.decoder(x)

#         return x

In [ ]:
len(dataset)

In [ ]:
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [17000,518])
_, pred_dataset = torch.utils.data.random_split(dataset, [,518])


In [ ]:
# del model
# del batch
# del criterion
# del masks
# del images
# del outputs
# import gc
# gc.collect()

In [ ]:
torch.cuda.empty_cache()
# gc.collect()

In [ ]:
def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
    # You can comment out this line if you are passing tensors of equal shape
    # But if you are passing output from UNet or something it will most probably
    # be with the BATCH x 1 x H x W shape
    outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    
    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    
    thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
    return thresholded  #

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Create an instance of the U-Net model
in_channels = 3  # Modify this based on the number of input channels in your images
out_channels = 1  # Modify this based on the number of classes or structures being segmented
model = smp.UnetPlusPlus(
    encoder_name="timm-resnest101e",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=4,                      # model output channels (number of classes in your dataset)
)
model = model.cuda()
# Define the loss function (e.g., binary cross-entropy)
criterion = smp.losses.DiceLoss('multiclass')#nn.CrossEntropyLoss()#BCEWithLogitsLoss()

# Define the optimizer (e.g., Adam)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 15
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()
    pbar = tqdm(train_dataloader)
    running_loss = 0.0
    for i, batch in enumerate(pbar):
        images = batch['image'].cuda()
        masks = batch['mask'].cuda()

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate the loss
        loss = criterion(outputs, masks.long())
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        tp, fp, fn, tn = smp.metrics.get_stats(torch.argmax(outputs, 1).unsqueeze(1).type(torch.int32), masks.type(torch.int32), mode='multiclass', num_classes = 3)

        # then compute metrics with required reduction (see metric docs)
        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        running_loss += loss.item()
        pbar.set_postfix({'loss':(running_loss/(i+1)), 'iou':iou_score})
    # Print training loss for the epoch
    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}")

    # Evaluation on the validation set
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            images = batch['image'].cuda()
            masks = batch['mask'].cuda()

            # Forward pass
            outputs = model(images)

            # Calculate the loss
            loss = criterion(outputs, masks)
            val_loss += loss.item()

    # Print validation loss for the epoch
    val_loss /= len(val_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}")


In [ ]:
pip install wandb

In [ ]:
wandb login


In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint


In [ ]:
import gc
gc.collect()

In [ ]:
import io
from pytorch_lightning.callbacks import LearningRateMonitor
lr_monitor = LearningRateMonitor(logging_interval='step')
# from pytorch_lightning.metrics.functional import accuracy

class Unetplusplus(pl.LightningModule):
    def __init__(self, encoder_name, encoder_weights, in_channels, classes, learning_rate):
        super(Unetplusplus, self).__init__()
        self.model = smp.UnetPlusPlus(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            in_channels=in_channels,
            classes=classes
        )
        self.criterion = smp.losses.DiceLoss('multiclass')
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, masks = batch['image'].float(), batch['mask']
        outputs = self(images)
        loss = self.criterion(outputs, masks.long())
        tp, fp, fn, tn = smp.metrics.get_stats(torch.argmax(outputs, 1).unsqueeze(1).type(torch.int32), masks.type(torch.int32), mode='multiclass', num_classes=3)
        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        self.log('train_loss', loss, prog_bar=True, logger=True)
        self.log('train_iou', iou_score, prog_bar=True, logger=True)
        del batch
        gc.collect()
        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch['image'].float(), batch['mask']
        outputs = self(images)
        loss = self.criterion(outputs, masks.long())
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=0.2)

        lr_scheduler_cont = {
            'scheduler':  torch.optim.lr_scheduler.ReduceLROnPlateau(
                                                                        optimizer,
                                                                        mode='min',
                                                                        factor=0.8,
                                                                        patience=200,
                                                                        threshold=1e-2,
                                                                        min_lr=1e-8,
                                                                        verbose=True
                                                                    ),
            'name': 'learning_rate',
            'monitor' : 'train_loss',
            "interval" : "step" 
        }
        return optimizer

# Create the Lightning DataModules
# train_dataloader = DataLoader(...)
# val_dataloader = DataLoader(...)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)
# Create the Lightning trainer
wandb_logger = WandbLogger()
checkpoint_callback = ModelCheckpoint(dirpath="/kaggle/working/ckpt/", save_top_k=3, monitor="val_loss")
trainer = pl.Trainer(max_epochs=15, devices=[0,1],accelerator="gpu",precision=16, logger=wandb_logger, callbacks=[checkpoint_callback])  # Set the appropriate number of GPUs

# Create the Lightning model
model = Unetplusplus(
    encoder_name="timm-resnest101e",
    encoder_weights="imagenet",
    in_channels=3,
    classes=4,
    learning_rate=1e-3
)

In [ ]:
model_ = smp.UnetPlusPlus(
    encoder_name="timm-resnest101e",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=4,                      # model output channels (number of classes in your dataset)
)

In [ ]:
model_.load_state_dict(torch.load('/kaggle/input/hubmap-unet-inst-model/model.ckpt'))

In [ ]:
trainer.fit(model, train_dataloader, val_dataloader)


In [ ]:
trainer.save_checkpoint("model.ckpt")

In [ ]:
torch.save(model.state_dict(),'/kaggle/working/model.pth' )

In [ ]:
torch.sum(torch.rand(4,1,512,512), dim=[1,2,3])/(512*512)

In [ ]:
pred_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)


In [ ]:
class Unetplusplus(pl.LightningModule):
    def __init__(self, encoder_name, encoder_weights, in_channels, classes, learning_rate):
        super(Unetplusplus, self).__init__()
        self.model = smp.UnetPlusPlus(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            in_channels=in_channels,
            classes=classes
        )
        self.criterion = smp.losses.DiceLoss('multiclass')
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, masks = batch['image'], batch['mask']
        outputs = self(images)
        loss = self.criterion(outputs, masks.long())
        tp, fp, fn, tn = smp.metrics.get_stats(torch.argmax(outputs, 1).unsqueeze(1).type(torch.int32), masks.type(torch.int32), mode='multiclass', num_classes=3)
        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
        self.log('train_loss', loss, prog_bar=True, logger=True)
        self.log('train_iou', iou_score, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch['image'], batch['mask']
        outputs = self(images)
        loss = self.criterion(outputs, masks.long())
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss
    def predict_step(self, batch, batch_idx):
        images, masks = batch['image'], batch['mask']
#         print(imah)
        output = self(images)
        return output
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [ ]:
model = Unetplusplus(
    encoder_name="timm-resnest101e",
    encoder_weights="imagenet",
    in_channels=3,
    classes=4,
    learning_rate=1e-4
)
model = model.load_from_checkpoint(checkpoint_path='/kaggle/input/hubmap-unet-inst-model/model.ckpt',
                                    encoder_name="timm-resnest101e",
                                    encoder_weights="imagenet",
                                    in_channels=3,
                                    classes=4,
                                    learning_rate=1e-4)
model.eval()

trainer = pl.Trainer(devices = [0],accelerator="gpu", inference_mode=True)    


In [ ]:
results = trainer.predict(model, pred_dataloader)

In [ ]:
results[0][:,1,:,:].shape

In [ ]:
# model.eval()
# val_loss = 0.0
# iou = 0.0
# for i in [0.6]:
#     with torch.no_grad():
for i,batch in enumerate(pred_dataloader):
    images = batch['image'].cuda()
    masks = batch['mask'].cuda()
    outputs = results[i][:,2,::]
#     # Forward pass
#     outputs = model.predict(images)

    # Calculate the loss
#     loss = criterion(outputs, maskslong())
#     val_loss += loss.item()
    
    sig_out = torch.sigmoid(outputs)
    images = images.detach().cpu().squeeze().numpy().transpose(1,2,0)
    masks = masks.detach().cpu().squeeze().numpy()
    sig_out = sig_out.detach().cpu().squeeze().numpy()
    overlay = np.copy(images)
    overlay[masks > 0] = [1, 0, 0]  # Set the mask pixels to red (adjust color as needed)
    overlay2 = np.copy(images)
    overlay2[sig_out > 0.2] = [1, 0, 0]  # Set the mask pixels to red (adjust color as needed)

    # Plot the image with the overlay
    fig, axes = plt.subplots(1, 3, figsize=(10, 5))
    axes[0].imshow(images)
    axes[0].set_title('Image')
    axes[1].imshow(overlay)
    axes[1].set_title('Image with Mask Overlay')
    axes[2].imshow(overlay2)
    axes[2].set_title('Image with pred Overlay')
    plt.show()
    if i>10:
        break
    #         print(sig_out[sig_out>0.5])
    #         break
#     tp, fp, fn, tn = smp.metrics.get_stats(outputs, masks.type(torch.int32).cuda(), mode='binary', threshold=i)
#     iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro").detach().cpu().numpy()
# #         print(f"Score:{torch.sum(outputs, dim=[1,2,3])/torch.count_nonzero(outputs)}, iou: {iou_score}")
#     iou += iou_score
# # Print validation loss for the epoch
# val_loss /= len(pred_dataloader)
# iou /= len(pred_dataloader)
# print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, iou : {iou} , thrshold: {i}")

In [ ]:
image_path = f"/kaggle/input/hubmap-hacking-the-human-vasculature/test/72e40acccadf.tif"
image = rasterio.open(image_path).read()
model2 = model.cuda()
pred = model2.predict_step({'image':(torch.from_numpy(np.expand_dims(image,axis=0)).float() / 255.0).cuda(), 'mask':np.array([])}, 0)[:,1,::]
pred = pred.detach().cpu()
sig_out = torch.sigmoid(pred).squeeze().numpy()
print(sig_out.shape)
overlay = np.copy(image.transpose(1,2,0))
overlay[sig_out>0.2] = [0, 255, 0]  # Set the mask pixels to red (adjust color as needed)

#         # Plot the image with the overlay
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image.transpose(1,2,0))
axes[0].set_title('Image')
axes[1].imshow(overlay)
axes[1].set_title('Image with Mask Overlay')
#         axes[2].imshow(overlay2)
#         axes[2].set_title('Image with pred Overlay')
plt.show()

In [ ]:
preds=[]
for file in os.listdir("/kaggle/input/hubmap-hacking-the-human-vasculature/test/"):
    image_path = f"/kaggle/input/hubmap-hacking-the-human-vasculature/test/{file}"
    image = rasterio.open(image_path).read()
    pred = torch.sigmoid(model(torch.from_numpy(np.expand_dims(image.astype(np.float32),axis=0)).cuda()))
    pred = pred.detach().cpu().squeeze().numpy()
    pred = encode_binary
    preds.append(pred)
# overlay = np.copy(image.transpose(1,2,0))
# overlay[pred > 0.3] = [1, 0, 0]  # Set the mask pixels to red (adjust color as needed)

# #         # Plot the image with the overlay
# fig, axes = plt.subplots(1, 2, figsize=(10, 5))
# axes[0].imshow(image.transpose(1,2,0))
# axes[0].set_title('Image')
# axes[1].imshow(overlay)
# axes[1].set_title('Image with Mask Overlay')
# #         axes[2].imshow(overlay2)
# #         axes[2].set_title('Image with pred Overlay')
# plt.show()

In [ ]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib


def encode_binary_mask(mask: np.ndarray) -> t.Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

In [ ]:
# 

# mask2former